# bmi.csv
- 딥러닝 모델을 만들어 보세요

- [178, 57] 사람의 label을 예측해라 thin
- [173, 68] 사람의 label을 예측해라 normal



In [5]:
import pandas as pd
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split

# 1.데이터 로드

In [4]:
df = pd.read_csv('../data/bmi.csv')
df.shape # 20000 x 3
df.head()

,height,weight,label
0,186,47,thin
1,143,74,fat
2,144,55,fat
3,173,60,normal
4,170,74,fat


In [16]:
df['label'].unique() # 종류 3개 -> activation function는 소프트맥스로 하는게 나아보임?

array(['thin', 'fat', 'normal'], dtype=object)

In [57]:
X = df.iloc[:,0:2]
y = df.iloc[:,2]
y = pd.get_dummies(y) # 분류니까 one hot 인코딩
y

,fat,normal,thin
0,0,0,1
1,1,0,0
2,1,0,0
3,0,1,0
4,1,0,0
...,...,...,...
19995,0,0,1
19996,0,0,1
19997,1,0,0
19998,0,1,0


In [60]:
X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=0.3, shuffle=True) # 무작위 섞음

In [61]:
X_train.shape # 14000 x 2
X_test.shape # 6000 x 2
y_train.shape # 14000 x 3

(14000, 3)

In [69]:
# 모델설정
model = Sequential()
model.add(Dense(24, input_shape=(X_train.shape[1],), activation='relu')) # 입력층 x 출력층 + bias = 
model.add(Dense(10, activation='relu')) # 입력층 x 출력층 + bias = 
model.add(Dense(3, activation='softmax')) # 입력층 x 출력층 + bias = 
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_15 (Dense)            (None, 24)                72        
                                                                 
 dense_16 (Dense)            (None, 10)                250       
                                                                 
 dense_17 (Dense)            (None, 3)                 33        
                                                                 
Total params: 355
Trainable params: 355
Non-trainable params: 0
_________________________________________________________________


In [70]:
# 모델 컴파일
model.compile(loss='categorical_crossentropy'
             ,optimizer='adam'
             ,metrics=['accuracy'])

In [72]:
# 모델 실행
history = model.fit(X_train, y_train, epochs=30, batch_size=5)

Epoch 1/30
2800/2800 [==============================] - 3s 898us/step - loss: 0.5285 - accuracy: 0.7501
Epoch 2/30
2800/2800 [==============================] - 2s 886us/step - loss: 0.4560 - accuracy: 0.7816
Epoch 3/30
2800/2800 [==============================] - 2s 879us/step - loss: 0.4098 - accuracy: 0.8044
Epoch 4/30
2800/2800 [==============================] - 2s 886us/step - loss: 0.3385 - accuracy: 0.8441
Epoch 5/30
2800/2800 [==============================] - 2s 889us/step - loss: 0.2719 - accuracy: 0.8806
Epoch 6/30
2800/2800 [==============================] - 3s 901us/step - loss: 0.2244 - accuracy: 0.9048
Epoch 7/30
2800/2800 [==============================] - 3s 898us/step - loss: 0.2046 - accuracy: 0.9107
Epoch 8/30
2800/2800 [==============================] - 2s 884us/step - loss: 0.1836 - accuracy: 0.9229
Epoch 9/30
2800/2800 [==============================] - 3s 904us/step - loss: 0.1698 - accuracy: 0.9279
Epoch 10/30
2800/2800 [==============================] - 2s 890u

# 테스트데이터 넣어보기

In [79]:
score = model.evaluate(X_test, y_test)
print(f'Test accuracy {score[1]:0.3}') # 정확도 0.978

188/188 [==============================] - 0s 736us/step - loss: 0.0666 - accuracy: 0.9778
Test accuracy 0.978


In [137]:
def get_answer(check_list):
    idx  = 0 # 첫번재 index
    prev = check_list[0] # 첫번째 값
    temp = check_list
    for i in range(1, len(check_list)):
        if prev < temp[i]:
            prev = temp[i]
            idx = i
    return y.columns[idx]


In [139]:
# 예측데이터 입력
h = []
h.append([178,57])
h.append([173,68])

ans = []
for i in h:
    a = get_answer(model.predict([i])[0])
    ans.append(a)
ans # ['thin', 'normal']

1/1 [==============================] - 0s 21ms/step


['thin', 'normal']

# 모델저장

In [84]:
from tensorflow.keras.models import load_model
model.save('model/실습-bmi.hdf5')